$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\ketbra}[2]{| #1\rangle \langle #2|} $
$ \newcommand{\tr}{{\rm tr}} $
$ \newcommand{\i}{{\color{blue} i}} $
$ \newcommand{\Hil}{{\cal H}} $
$ \newcommand{\V}{{\cal V}} $
$ \newcommand{\bn}{{\bf n}} $


# 3. Qubit spectroscopy

Pulses give grain-level access to the transmon dynamics, like excitation to higher energy states. In this notebook we'll see how the qubit responds to pulses with different frequencies, exploring some observable fenemona at Qmio.


## 3.1. Measuring the anharmonicity

When we studied pulses for gate implementation, made a series of assumtions: 

- We only considered the two computational energy levels of the transmon.

- We set the driving frequency at the resonance of the qubit.

But in reality, transmons have higher energy levels that can be accessed by microwave pulses. For example, the second excited state $\ket{2}$ can be reached if we know the anharmonicity $\alpha$ of the qubit. We could apply a pulse of frequency $\omega_q+\alpha$ to promote from $\ket{1}$ to $\ket{2}$, or one with frequency $2\omega_q+\alpha$ to promote directly from $\ket{0}$. But, how can we measure these higher energy states? These states also produce a shift in the energy levels of the driven resonator, inducing a distinct observable response. When working with IQ-mixing readout, this would result in clouds on the IQ plane different from $\ket{0}$ and $\ket{1}$.<br>
<br>

Unfortunatetly, in Qmio we don't have access to these IQ measurements, only to their projection to the $\ket{0}$-$\ket{1}$ bisector. This is a problem especially for the case of $\ket{2}$, as its cloud lands right above $\ket{1}$, making them practically undistinguishable from their bisector projection. This means that we can't directly measure $\omega_{12}$ or even $\omega_{02}$, as it would lead to a high error from the readout. Nevertheless, there is another effect that we can use to measure the anharmonicity.<br>
<br>

The **two-photon absorption (TPA)** is the simultaneous absorption of two photons -with or without identical frequencies- where the system is promoted through a virtual energy level. In our case, this effect can promote the system from $\ket{0}$ to $\ket{2}$ with photons of frequency $\omega_q+\alpha/2$. Why is the convenient? First, we don't need to access the first excited state to prometo to $\ket{2}$, so we can interpret all the measurements in the bisector as the $\ket{2}$ with minimal readout error. Second, these frequency is close to the operative frequency of the qubit, much more convenient than the one for the one-photon $\ket{0}$-$\ket{2}$ transition. What we need to keep in mind is that TPA depends on the simultaneous absorption of two photons, reducing its probability compared to one-photon absorptions, which would lead to a less populated $\ket{2}$.


## 3.2. Spectroscopy with OpenPulse

A qubit spectroscopy consists of sweeping the driving frequencies for an X gate calibrated pulse and observing the response. The expected behaviour is a peak around the resonance frequency, corresponding to the $\ket{0}$-$\ket{1}$ transition. To change the pulses' frequency, we must do it by modifying the driving frame. In OpenPulse, this can be achieved with two functions:

- **```set_frequency(frame, float)```**: sets the frequency to some value.

- **```shift_frequency(frame, float)```**: shift the frequency by some value.

This functions can only be invoked in calibration blocks (```cal``` and ```defcal```).<br>
<br>


In [ ]:
from qmio import QmioRuntimeService
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

service = QmioRuntimeService()

def run_instruction(_backend, _instruction, _shots):  
        _res = _backend.run(circuit = _instruction, shots = _shots, res_format = 'raw')     
        try :
            return _res["results"].reshape((_shots,))                                      
        except :
            print(_res) 

In [ ]:
qubit = 8

# Saturation pulse setting
amplitude = 0.04646
duration = 500 # dt
sigma = 100 # dt

def set_freq(_freq):
    _inst = f'''OPENQASM 3;
    defcalgrammar "openpulse";

    cal {{
        extern frame q{qubit}_drive;
        set_frequency(q{qubit}_drive, {_freq});
        waveform wf = gaussian({amplitude}, {duration}dt, {sigma}dt);
    }}

    defcal custom_pulse ${qubit} {{
        play(q{qubit}_drive, wf);
    }}

    custom_pulse ${qubit};
    measure ${qubit};'''
    return _inst

In [ ]:
# Frequency sweep settings
freq_start = 4.25e9
freq_end = 4.45e9
freq_step = 0.001e9
freq_sweep = np.arange(freq_start, freq_end, freq_step)

shots = 1000

results = []
with service.backend(name = "qpu") as backend:
    for f in freq_sweep:
        instruction = set_freq(f)
        res = run_instruction(backend, instruction, shots)
        results.append(len(np.where(res>0)[0])/shots)

In [ ]:
plt.plot(freq_sweep, results)
plt.ylabel(r'$\ket{0}$ counts')
plt.xlabel('Frequency (Hz)')
plt.show()